<a href="https://colab.research.google.com/github/t4toast/project-one/blob/main/Sands_project_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project one, final
- Diane T Sands

-steps to remember:
1. import all libraries
2. Load the dataset
3. make copy of df
5. check about dataset info
6. df.isna().sum().sum()
7. check for duplicates and drop the duplicates - df.duplicated().sum()
8. check for incosistacneis
9. Split the dataset-train/test
10. column selection - separate the categorical and numerical one data
11. handling missing values using imputer for categorical and numerical dataset
12. justify the strategy used in code like why are you using mean, median or mode.
13. Make pipleines
14. use column transformers for scaling , ohe, for the categorical and numerical data.
15.fit the data to train and transform train/test both


In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree
from sklearn import set_config
set_config(display='diagram')

In [87]:
#load the dataset
filename = '/content/drive/MyDrive/Colab Notebooks/sales_predictions.csv'
df = pd.read_csv(filename)

In [88]:
#make a copy of the data
ml_df = df.copy()

In [89]:
#explore the data some 
ml_df. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [90]:
#check for missing data
ml_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [91]:
#check for duplicates
ml_df.duplicated().sum()

0

In [92]:
#no duplicated rows, so no need to drop anything.
#Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target 
#and the rest of the relevant variables as your features matrix.  
X = ml_df.drop('Item_Outlet_Sales', axis =1)
y = ml_df['Item_Outlet_Sales']

In [93]:
#Perform a train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
X_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,NCG06,16.350,Low Fat,0.029565,Household,256.4646,OUT018,2009,Medium,Tier 3,Supermarket Type2
7510,FDV57,15.250,Regular,0.000000,Snack Foods,179.7660,OUT018,2009,Medium,Tier 3,Supermarket Type2
5828,FDM27,12.350,Regular,0.158716,Meat,157.2946,OUT049,1999,Medium,Tier 1,Supermarket Type1
5327,FDG24,7.975,Low Fat,0.014628,Baking Goods,82.3250,OUT035,2004,Small,Tier 2,Supermarket Type1
4810,FDD05,19.350,Low Fat,0.016645,Frozen Foods,120.9098,OUT045,2002,NaN,Tier 2,Supermarket Type1


In [94]:
#drop item_identifier column
ml_df.drop('Item_Identifier', axis=1)

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [95]:
#Create a ColumnTransformer to preprocess the data. 
cat_selector = make_column_selector(dtype_include= 'object')
num_selector = make_column_selector(dtype_include= 'number')
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown= 'ignore')
#make tuples
num_tuple = (scaler, num_selector)
cat_tuple = (ohe, cat_selector)

col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder = 'passthrough')
col_transformer.fit(X_train)
X_train_processed = col_transformer.transform(X_train)
X_test_processed = col_transformer.transform(X_test)

In [96]:
#Make sure your imputation of missing values occurs  after the train test split using SimpleImputer.  
#Item Weight and Outlet Size are the two categories with missing data.
#look again at the data...

ml_df['Item_Weight'].describe()  

count    7060.000000
mean       12.857645
std         4.643456
min         4.555000
25%         8.773750
50%        12.600000
75%        16.850000
max        21.350000
Name: Item_Weight, dtype: float64

In [97]:
ml_df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [98]:
ml_df.dropna(subset=['Item_Outlet_Sales'], inplace=True)

In [99]:
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='drop')

mean_imputer = SimpleImputer(strategy='mean')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')
#using mean for Item Weight as it is very close to 50% of the data



In [100]:
#make pipelines
number_pipe = make_pipeline(mean_imputer, scaler)
number_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [101]:
nominal_pipe = make_pipeline(constant_imputer, ohe)
nominal_pipe

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='drop', sparse=False))])

In [102]:
#create tuples to go with pipelines
num_selector = make_column_selector(dtype_include='number')
nom_cols = ['Outlet_Establishment_Year']


number_tuple = (number_pipe, num_selector)
nominal_tuple = (nominal_pipe, nom_cols)


In [103]:
#preprocessor
preprocessor = make_column_transformer(nominal_tuple,
                                       number_tuple,
                                       remainder='drop')

In [104]:
#define the MAE, MSE, RMSE and R2 series of code to use later
def evaluate_regression(y_true, y_pred):
  
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_true, y_pred)

  print(f'scores: \nMAE: {mae:,.2f} \nMSE: {mse:,.2f} \nRMSE: {rmse:,.2f} \nR2: {r2:.2f}')


In [105]:
#linear regression model
lin_reg = LinearRegression()

lin_reg_pipe = make_pipeline(preprocessor, lin_reg)

lin_reg_pipe.fit(X_train, y_train)

ValueError: ignored

In [ ]:
#run evaluation parameters
print('Training')
evaluate_regression(y_train, dec_tree_pipe.predict(X_train))
print('Testing')
evaluate_regression(y_test, dec_tree_pipe.predict(X_test))

In [ ]:
#do a decision tree
dec_tree = DecisionTreeRegressor(random_state = 42)
dec_tree.fit(X_train, y_train)


In [ ]:
#run evaluation parameters
print('Training')
evaluate_regression(y_train, dec_tree_pipe.predict(X_train))
print('Testing')
evaluate_regression(y_test, dec_tree_pipe.predict(X_test))

I have have tried 2 different models on the data set. I need to determine which model to implement.

Overall, which model do you recommend? >>> neither yet since neither of them worked.
Justify your recommendation.